In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random

# Loading training

In [30]:
datadir='/Users/molins/Desktop/final-project/input/chest_xray/train'
categories=['NORMAL','PNEUMONIA']
IMG_SIZE=200

training_data=[]
def create_training_data(datadir):
    categories = ['NORMAL', 'PNEUMONIA']
    for category in categories:
        path = os.path.join(datadir, category)  # path to normal or pneumonia
        class_num=categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array,class_num])
            except Exception as e:
                print('Error: {}'.format(e))

create_training_data('/Users/molins/Desktop/final-project/input/chest_xray/train')

Error: OpenCV(4.1.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:3718: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error: OpenCV(4.1.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:3718: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



In [31]:
print(len(training_data))

5216


In [32]:
random.shuffle(training_data)

In [33]:
for sample in training_data[:10]:
    print(sample[1])

1
1
0
1
1
0
1
1
1
1


In [34]:
X=[]
y=[]

In [35]:
for features,label in training_data:
    X.append(features)
    y.append(label)

In [36]:
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [37]:
import pickle

pickle_out = open('X.pickle','wb')
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open('y.pickle','wb')
pickle.dump(y, pickle_out)
pickle_out.close()

In [38]:
pickle_in=open('X.pickle','rb')
X_load=pickle.load(pickle_in)

In [39]:
len(X_load)

5216